## makemore: becoming a backprop ninja

In this chapter we calculate the backprogations manually by hand. Although inputs/outputs of each layer are tensors the principles of micrograd are still the same. For each item we look at the elements it influences and we can get a vectorized formula to get the gradient of the whole tensor.

If we know the mathematical formula (like in cross_entropy), then we can skip backpropagating each step and just get the derivative using calculus and use it.

In [54]:
# there no change change in the first several cells from last lecture

In [55]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt # for making figures
%matplotlib inline

In [56]:
# download the names.txt file from github
!wget https://raw.githubusercontent.com/karpathy/makemore/master/names.txt

zsh:1: command not found: wget


In [57]:
# read in all the words
words = open('names.txt', 'r').read().splitlines()
print(len(words))
print(max(len(w) for w in words))
print(words[:8])

32033
15
['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']


In [58]:
# build the vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
vocab_size = len(itos)
print(itos)
print(vocab_size)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}
27


In [59]:
# build the dataset
block_size = 3 # context length: how many characters do we take to predict the next one?

def build_dataset(words):
  X, Y = [], []

  for w in words:
    context = [0] * block_size
    for ch in w + '.':
      ix = stoi[ch]
      X.append(context)
      Y.append(ix)
      context = context[1:] + [ix] # crop and append

  X = torch.tensor(X)
  Y = torch.tensor(Y)
  print(X.shape, Y.shape)
  return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr,  Ytr  = build_dataset(words[:n1])     # 80%
Xdev, Ydev = build_dataset(words[n1:n2])   # 10%
Xte,  Yte  = build_dataset(words[n2:])     # 10%

torch.Size([182625, 3]) torch.Size([182625])
torch.Size([22655, 3]) torch.Size([22655])
torch.Size([22866, 3]) torch.Size([22866])


In [60]:
# ok biolerplate done, now we get to the action:

In [61]:
# utility function we will use later when comparing manual gradients to PyTorch gradients
def cmp(s, dt, t):
  ex = torch.all(dt == t.grad).item()
  app = torch.allclose(dt, t.grad)
  maxdiff = (dt - t.grad).abs().max().item()
  print(f'{s:15s} | exact: {str(ex):5s} | approximate: {str(app):5s} | maxdiff: {maxdiff}')

In [62]:
n_embd = 10 # the dimensionality of the character embedding vectors
n_hidden = 64 # the number of neurons in the hidden layer of the MLP

g = torch.Generator().manual_seed(2147483647) # for reproducibility
C  = torch.randn((vocab_size, n_embd),            generator=g)
# Layer 1
W1 = torch.randn((n_embd * block_size, n_hidden), generator=g) * (5/3)/((n_embd * block_size)**0.5)
b1 = torch.randn(n_hidden,                        generator=g) * 0.1 # using b1 just for fun, it's useless because of BN
# Layer 2
W2 = torch.randn((n_hidden, vocab_size),          generator=g) * 0.1
b2 = torch.randn(vocab_size,                      generator=g) * 0.1
# BatchNorm parameters
bngain = torch.randn((1, n_hidden))*0.1 + 1.0
bnbias = torch.randn((1, n_hidden))*0.1

# Note: I am initializating many of these parameters in non-standard ways
# because sometimes initializating with e.g. all zeros could mask an incorrect
# implementation of the backward pass.

parameters = [C, W1, b1, W2, b2, bngain, bnbias]
print(sum(p.nelement() for p in parameters)) # number of parameters in total
for p in parameters:
  p.requires_grad = True

4137


In [63]:
batch_size = 32
n = batch_size # a shorter variable also, for convenience
# construct a minibatch
ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
Xb, Yb = Xtr[ix], Ytr[ix] # batch X,Y

In [64]:
# forward pass, "chunkated" into smaller steps that are possible to backward one at a time

emb = C[Xb] # embed the characters into vectors
embcat = emb.view(emb.shape[0], -1) # concatenate the vectors
# Linear layer 1
hprebn = embcat @ W1 + b1 # hidden layer pre-activation
# BatchNorm layer
bnmeani = 1/n*hprebn.sum(0, keepdim=True)
bndiff = hprebn - bnmeani
bndiff2 = bndiff**2
bnvar = 1/(n-1)*(bndiff2).sum(0, keepdim=True) # note: Bessel's correction (dividing by n-1, not n)
bnvar_inv = (bnvar + 1e-5)**-0.5
bnraw = bndiff * bnvar_inv
hpreact = bngain * bnraw + bnbias
# Non-linearity
h = torch.tanh(hpreact) # hidden layer
# Linear layer 2
logits = h @ W2 + b2 # output layer
# cross entropy loss (same as F.cross_entropy(logits, Yb))
logit_maxes = logits.max(1, keepdim=True).values
norm_logits = logits - logit_maxes # subtract max for numerical stability
counts = norm_logits.exp()
counts_sum = counts.sum(1, keepdims=True)
counts_sum_inv = counts_sum**-1 # if I use (1.0 / counts_sum) instead then I can't get backprop to be bit exact...
probs = counts * counts_sum_inv
logprobs = probs.log()
loss = -logprobs[range(n), Yb].mean()

# PyTorch backward pass
for p in parameters:
  p.grad = None
for t in [logprobs, probs, counts, counts_sum, counts_sum_inv, # afaik there is no cleaner way
          norm_logits, logit_maxes, logits, h, hpreact, bnraw,
         bnvar_inv, bnvar, bndiff2, bndiff, hprebn, bnmeani,
         embcat, emb]:
  t.retain_grad()
loss.backward()
loss

tensor(3.3501, grad_fn=<NegBackward0>)

In [53]:
# Exercise 1: backprop through the whole thing manually,
# backpropagating through exactly all of the variables
# as they are defined in the forward pass above, one by one

# -----------------
# YOUR CODE HERE :)
# -----------------
# logprobs.shape = (batch_size, num_classes) = (32, 27)
dlogprobs = torch.zeros(logprobs.shape)
dlogprobs[range(n), Yb] = -1/n
cmp('logprobs', dlogprobs, logprobs)

# dropbs.shape = (32, 27)
dprobs = dlogprobs / probs
cmp('probs', dprobs, probs)

# counts_sum_inv.shape = (32, 1)
dcounts_sum_inv = (dprobs * counts).sum(1, keepdims=True)
cmp('counts_sum_inv', dcounts_sum_inv, counts_sum_inv)

# counts_sum.shape = (32, 1)
dcounts_sum = -dcounts_sum_inv / (counts_sum ** 2)
cmp('counts_sum', dcounts_sum, counts_sum)

# counts.shape == (32, 27)
dcounts = torch.zeros(counts.shape) + dcounts_sum
dcounts += dprobs * counts_sum_inv
cmp('counts', dcounts, counts)

# norm_logits.shape = (32, 27)
dnorm_logits = dcounts * counts
cmp('norm_logits', dnorm_logits, norm_logits)

# logit_maxes.shape = (32, 1)
dlogit_maxes = -dnorm_logits.sum(1, keepdim=True)
cmp('logit_maxes', dlogit_maxes, logit_maxes)

# logits.shape = (32, 27)
dlogits = torch.zeros(logits.shape)
dlogits_max_indices = logits.max(1).indices
dlogits[torch.arange(dlogits.shape[0]), dlogits_max_indices] = dlogit_maxes[:,0]
dlogits += dnorm_logits
cmp('logits', dlogits, logits)

# h.shape = (32, 64), it's an output of hidden layer
dh = torch.zeros(h.shape)
dh = dlogits @ W2.T
cmp('h', dh, h)

# W2.shape = (64, 27)
dW2 = (dlogits.T @ h).T
cmp('W2', dW2, W2)

# b2.shape = (27)
db2 = dlogits.sum(dim=0)
cmp('b2', db2, b2)

# hpreact.shape = (32, 64)
dhpreact = (1 - h ** 2) * dh
cmp('hpreact', dhpreact, hpreact)

# bngain.shape = (1, 64)
dbngain = (dhpreact * bnraw).sum(0, keepdim=True)
cmp('bngain', dbngain, bngain)

# bnbias.shape = (1, 64)
dbnbias = dhpreact.sum(0, keepdim=True)
cmp('bnbias', dbnbias, bnbias)

# bnraw.shape = (32, 64)
dbnraw = dhpreact * bngain
cmp('bnraw', dbnraw, bnraw)

# bnvar_inv.shape = (1, 64), bndiff.shape = (32, 64)
dbnvar_inv = (dbnraw * bndiff).sum(0, keepdim=True)
cmp('bnvar_inv', dbnvar_inv, bnvar_inv)

# bnvar.shape = (1, 64)
dbnvar = -0.5 * dbnvar_inv * (bnvar + 1e-5) ** (-1.5)
cmp('bnvar', dbnvar, bnvar)

# bndiff2.shape = (32, 64)
dbndiff2 = torch.zeros(bndiff2.shape)
dbndiff2 = dbnvar / (n - 1)
cmp('bndiff2', dbndiff2, bndiff2)

# bndiff.shape = (32, 64)
dbndiff = 2 * bndiff * dbndiff2
dbndiff += dbnraw * bnvar_inv
cmp('bndiff', dbndiff, bndiff)

# bnmeani.shape = (1, 64), hprebn.shape = (32, 64)
dbnmeani = -dbndiff.sum(0, keepdim=True)
cmp('bnmeani', dbnmeani, bnmeani)

# hprebn.shape = (32, 64)
dhprebn = dbndiff
dhprebn += dbnmeani / n
cmp('hprebn', dhprebn, hprebn)

# embcat.shape = (32, 30), W1.shape = (30, 64), b1.shape = (64)
dembcat = dhprebn @ W1.T
cmp('embcat', dembcat, embcat)

# W1.shape = (30, 64)
dW1 = (dhprebn.T @ embcat).T
cmp('W1', dW1, W1)

# b1.shape = (64)
db1 = dhprebn.sum(0, keepdim=False)
print(f'{db1 == 0=}')
cmp('b1', db1, b1)

# emb.shape = (32, 3, 10)
demb = dembcat.view(emb.shape)
cmp('emb', demb, emb)

# C.shape = (27, 10), Xb.shape = (32, 3)
dC = torch.zeros(C.shape)
dC.index_add_(0, Xb.view(-1), demb.view(-1, 10))
cmp('C', dC, C)

logprobs        | exact: False | approximate: False | maxdiff: 0.03125
probs           | exact: False | approximate: False | maxdiff: 3.084042549133301
counts_sum_inv  | exact: False | approximate: False | maxdiff: 429.4279479980469
counts_sum      | exact: False | approximate: False | maxdiff: 5.917099475860596


/var/folders/t1/9zl0db0d4xg6nynn6pg2mgt80000gn/T/ipykernel_28562/2706152459.py:3: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/build/aten/src/ATen/core/TensorBody.h:491.)
  ex = torch.all(dt == t.grad).item()


TypeError: all(): argument 'input' (position 1) must be Tensor, not bool

In [48]:
# Exercise 2: backprop through cross_entropy but all in one go
# to complete this challenge look at the mathematical expression of the loss,
# take the derivative, simplify the expression, and just write it out

# forward pass

# before:
# logit_maxes = logits.max(1, keepdim=True).values
# norm_logits = logits - logit_maxes # subtract max for numerical stability
# counts = norm_logits.exp()
# counts_sum = counts.sum(1, keepdims=True)
# counts_sum_inv = counts_sum**-1 # if I use (1.0 / counts_sum) instead then I can't get backprop to be bit exact...
# probs = counts * counts_sum_inv
# logprobs = probs.log()
# loss = -logprobs[range(n), Yb].mean()

# now:
loss_fast = F.cross_entropy(logits, Yb)
print(loss_fast.item(), 'diff:', (loss_fast - loss).item())

3.361903429031372 diff: 2.384185791015625e-07


In [49]:
# backward pass

dlogits = None
counts = logits.exp()
sums = counts.sum(1, keepdim=True)
sums_except_for_me = sums - counts
# when logit is not picked, f(x) = (-1/n) * (picked_in_a_row / (e ** x + sums_except_for_me))
# dlogits is just a derivative of that
dlogits = (1 / n) * counts / sums
# when logit is picked, f(x) = (-1/n) * (e ** x / (e ** x + sums_except_for_me))
# dlogits is just a derivative of that
dlogits[torch.arange(n), Yb] = ((-1/n) * (sums_except_for_me / sums))[torch.arange(n), Yb]

# logits.shape = (32, 27)
cmp('logits', dlogits, logits) # I can only get approximate to be true, my maxdiff is 6e-9

logits          | exact: False | approximate: True  | maxdiff: 7.450580596923828e-09


In [50]:
# Exercise 3: backprop through batchnorm but all in one go
# to complete this challenge look at the mathematical expression of the output of batchnorm,
# take the derivative w.r.t. its input, simplify the expression, and just write it out
# BatchNorm paper: https://arxiv.org/abs/1502.03167

# forward pass

# before:
# bnmeani = 1/n*hprebn.sum(0, keepdim=True)
# bndiff = hprebn - bnmeani
# bndiff2 = bndiff**2
# bnvar = 1/(n-1)*(bndiff2).sum(0, keepdim=True) # note: Bessel's correction (dividing by n-1, not n)
# bnvar_inv = (bnvar + 1e-5)**-0.5
# bnraw = bndiff * bnvar_inv
# hpreact = bngain * bnraw + bnbias

# now:
hpreact_fast = bngain * (hprebn - hprebn.mean(0, keepdim=True)) / torch.sqrt(hprebn.var(0, keepdim=True, unbiased=True) + 1e-5) + bnbias
print('max diff:', (hpreact_fast - hpreact).abs().max())

max diff: tensor(4.7684e-07, grad_fn=<MaxBackward1>)


In [51]:
# backward pass

# before we had:
# dbnraw = bngain * dhpreact
# dbndiff = bnvar_inv * dbnraw
# dbnvar_inv = (bndiff * dbnraw).sum(0, keepdim=True)
# dbnvar = (-0.5*(bnvar + 1e-5)**-1.5) * dbnvar_inv
# dbndiff2 = (1.0/(n-1))*torch.ones_like(bndiff2) * dbnvar
# dbndiff += (2*bndiff) * dbndiff2
# dhprebn = dbndiff.clone()
# dbnmeani = (-dbndiff).sum(0)
# dhprebn += 1.0/n * (torch.ones_like(hprebn) * dbnmeani)

# calculate dhprebn given dhpreact (i.e. backprop through the batchnorm)
# (you'll also need to use some of the variables from the forward pass up above)

# -----------------
# hprebn.shape = (32, 64), hpreact.shape = (32, 64)
new_dbnvar = -(bngain / (2 * (bnvar + 1e-5) ** 1.5)) * (dhpreact * (hprebn - bnmeani)).sum(0, keepdim=True)
new_dbnmean = -bngain * bnvar_inv * dhpreact.sum(0, keepdim=True) + new_dbnvar * 2 / (n - 1) * (bnmeani - hprebn).sum(0, keepdim=True)
dhprebn = dhpreact * bngain * bnvar_inv +  new_dbnvar * 2 * (hprebn - bnmeani) / (n - 1) + new_dbnmean / n
# -----------------
    
cmp('hprebn', dhprebn, hprebn) # I can only get approximate to be true, my maxdiff is 9e-10

hprebn          | exact: False | approximate: True  | maxdiff: 1.1641532182693481e-09


In [52]:
# Exercise 4: putting it all together!
# Train the MLP neural net with your own backward pass

# init
n_embd = 10 # the dimensionality of the character embedding vectors
n_hidden = 200 # the number of neurons in the hidden layer of the MLP

g = torch.Generator().manual_seed(2147483647) # for reproducibility
C  = torch.randn((vocab_size, n_embd),            generator=g)
# Layer 1
W1 = torch.randn((n_embd * block_size, n_hidden), generator=g) * (5/3)/((n_embd * block_size)**0.5)
b1 = torch.randn(n_hidden,                        generator=g) * 0.1
# Layer 2
W2 = torch.randn((n_hidden, vocab_size),          generator=g) * 0.1
b2 = torch.randn(vocab_size,                      generator=g) * 0.1
# BatchNorm parameters
bngain = torch.randn((1, n_hidden))*0.1 + 1.0
bnbias = torch.randn((1, n_hidden))*0.1

parameters = [C, W1, b1, W2, b2, bngain, bnbias]
print(sum(p.nelement() for p in parameters)) # number of parameters in total
for p in parameters:
  p.requires_grad = True

# same optimization as last time
max_steps = 200000
batch_size = 32
n = batch_size # convenience
lossi = []

# use this context manager for efficiency once your backward pass is written (TODO)
#with torch.no_grad():

# kick off optimization
for i in range(max_steps):

  # minibatch construct
  ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
  Xb, Yb = Xtr[ix], Ytr[ix] # batch X,Y

  # forward pass
  emb = C[Xb] # embed the characters into vectors
  embcat = emb.view(emb.shape[0], -1) # concatenate the vectors
  # Linear layer
  hprebn = embcat @ W1 + b1 # hidden layer pre-activation
  # BatchNorm layer
  # -------------------------------------------------------------
  bnmean = hprebn.mean(0, keepdim=True)
  bnvar = hprebn.var(0, keepdim=True, unbiased=True)
  bnvar_inv = (bnvar + 1e-5)**-0.5
  bnraw = (hprebn - bnmean) * bnvar_inv
  hpreact = bngain * bnraw + bnbias
  # -------------------------------------------------------------
  # Non-linearity
  h = torch.tanh(hpreact) # hidden layer
  logits = h @ W2 + b2 # output layer
  loss = F.cross_entropy(logits, Yb) # loss function

  # backward pass
  for p in parameters:
    p.grad = None
  loss.backward() # use this for correctness comparisons, delete it later!

  # manual backprop! #swole_doge_meme
  # -----------------
  dlogits = None
  counts = logits.exp()
  sums = counts.sum(1, keepdim=True)
  sums_except_for_me = sums - counts
  # when logit is not picked, f(x) = (-1/n) * (picked_in_a_row / (e ** x + sums_except_for_me))
  # dlogits is just a derivative of that
  dlogits = (1 / n) * counts / sums
  # when logit is picked, f(x) = (-1/n) * (e ** x / (e ** x + sums_except_for_me))
  # dlogits is just a derivative of that
  dlogits[torch.arange(n), Yb] = ((-1/n) * (sums_except_for_me / sums))[torch.arange(n), Yb]
  dh = torch.zeros(h.shape)
  dh = dlogits @ W2.T
  dhpreact = (1 - h ** 2) * dh
  dC = torch.zeros(C.shape)
  dC.index_add_(0, Xb.view(-1), demb.view(-1, 10))
  bnmeani = bnmean
  new_dbnvar = -(bngain / (2 * (bnvar + 1e-5) ** 1.5)) * (dhpreact * (hprebn - bnmeani)).sum(0, keepdim=True)
  new_dbnmean = -bngain * bnvar_inv * dhpreact.sum(0, keepdim=True) + new_dbnvar * 2 / (n - 1) * (bnmeani - hprebn).sum(0, keepdim=True)
  dhprebn = dhpreact * bngain * bnvar_inv +  new_dbnvar * 2 * (hprebn - bnmeani) / (n - 1) + new_dbnmean / n
  dW1 = (dhprebn.T @ embcat).T
  db1 = dhprebn.sum(0, keepdim=False)
  dW2 = (dlogits.T @ h).T
  db2 = dlogits.sum(dim=0)
  dbngain = (dhpreact * bnraw).sum(0, keepdim=True)
  dbnbias = dhpreact.sum(0, keepdim=True)
  grads = [dC, dW1, db1, dW2, db2, dbngain, dbnbias]
  # -----------------

  # update
  lr = 0.1 if i < 100000 else 0.01 # step learning rate decay
  for p, grad in zip(parameters, grads):
    #p.data += -lr * p.grad # old way of cheems doge (using PyTorch grad from .backward())
    p.data += -lr * grad # new way of swole doge TODO: enable

  # track stats
  if i % 10000 == 0: # print every once in a while
    print(f'{i:7d}/{max_steps:7d}: {loss.item():.4f}')
  lossi.append(loss.log10().item())

12297
      0/ 200000: 3.8140
  10000/ 200000: 2.2180
  20000/ 200000: 2.2067
  30000/ 200000: 2.4489
  40000/ 200000: 1.9965
  50000/ 200000: 2.4747
  60000/ 200000: 2.4296
  70000/ 200000: 2.0654
  80000/ 200000: 2.3846
  90000/ 200000: 2.1710


KeyboardInterrupt: 

In [ ]:
# useful for checking your gradients
# for p,g in zip(parameters, grads):
#   cmp(str(tuple(p.shape)), g, p)

In [ ]:
# calibrate the batch norm at the end of training

with torch.no_grad():
  # pass the training set through
  emb = C[Xtr]
  embcat = emb.view(emb.shape[0], -1)
  hpreact = embcat @ W1 + b1
  # measure the mean/std over the entire training set
  bnmean = hpreact.mean(0, keepdim=True)
  bnvar = hpreact.var(0, keepdim=True, unbiased=True)


In [ ]:
# evaluate train and val loss

@torch.no_grad() # this decorator disables gradient tracking
def split_loss(split):
  x,y = {
    'train': (Xtr, Ytr),
    'val': (Xdev, Ydev),
    'test': (Xte, Yte),
  }[split]
  emb = C[x] # (N, block_size, n_embd)
  embcat = emb.view(emb.shape[0], -1) # concat into (N, block_size * n_embd)
  hpreact = embcat @ W1 + b1
  hpreact = bngain * (hpreact - bnmean) * (bnvar + 1e-5)**-0.5 + bnbias
  h = torch.tanh(hpreact) # (N, n_hidden)
  logits = h @ W2 + b2 # (N, vocab_size)
  loss = F.cross_entropy(logits, y)
  print(split, loss.item())

split_loss('train')
split_loss('val')

In [ ]:
# I achieved:
# train 2.0718822479248047
# val 2.1162495613098145

In [ ]:
# sample from the model
g = torch.Generator().manual_seed(2147483647 + 10)

for _ in range(20):

    out = []
    context = [0] * block_size # initialize with all ...
    while True:
      # forward pass
      emb = C[torch.tensor([context])] # (1,block_size,d)
      embcat = emb.view(emb.shape[0], -1) # concat into (N, block_size * n_embd)
      hpreact = embcat @ W1 + b1
      hpreact = bngain * (hpreact - bnmean) * (bnvar + 1e-5)**-0.5 + bnbias
      h = torch.tanh(hpreact) # (N, n_hidden)
      logits = h @ W2 + b2 # (N, vocab_size)
      # sample
      probs = F.softmax(logits, dim=1)
      ix = torch.multinomial(probs, num_samples=1, generator=g).item()
      context = context[1:] + [ix]
      out.append(ix)
      if ix == 0:
        break

    print(''.join(itos[i] for i in out))